In [7]:
!mpirun /usr/bin/python3.7 /home/nest_elephant_tvb/simulation/run_exploration.py ./test_1/parameter.py

[INFO] [2020.6.24 7:23:26 /home/nest-io-dev/nestkernel/rng_manager.cpp:208 @ Network::create_rngs_] : Creating default RNGs
[INFO] [2020.6.24 7:23:26 /home/nest-io-dev/nestkernel/rng_manager.cpp:251 @ Network::create_grng_] : Creating new default global RNG

              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: UNKNOWN
 Built: Jun 24 2020 04:40:50

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.

2020-06-24 07:23:27,621 - WARNING - tvb.simulator.common - psutil module not available: no warnings will be issued when a
    simulation may require more memory than available
   INFO  log level set to INFO
time: 2020-06-24 07:23:27.979690 BEGIN SIMULATION 

Time to initialize the network: 0.02 s
[0, 800]
[800, 1000]
2020-06-24 07:23:28,768 - WARNING - tvb.simulator.common - psutil module not available: no 

/usr/bin/mpirun
